In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Images.zip', 'r') 
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
import os
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle

In [ ]:
class_names = ['birds','drones','helicopters','jets','planes']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)
print(class_names_label)

IMAGE_SIZE = (300,300)

{'birds': 0, 'drones': 1, 'helicopters': 2, 'jets': 3, 'planes': 4}


In [ ]:
def load_data():
  DIRECTORY = r"/tmp/Images"
  CATEGORY = ['Train','Test']

  output = []

  for category in CATEGORY:
    path = os.path.join(DIRECTORY,category)
    images = []
    labels = []

    print("Loading {}".format(category))

    for folder in os.listdir(path):
      label = class_names_label[folder]

      for file in os.listdir(os.path.join(path,folder)):
        img_path = os.path.join(os.path.join(path,folder),file)

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,IMAGE_SIZE)

        images.append(img)
        labels.append(label)

    images = np.array(images,dtype = 'float32')
    lables = np.array(labels,dtype = 'int32')

    output.append((images,lables))
  
  return output



In [ ]:
(train_images,train_labels) , (test_images,test_labels) = load_data()

Loading Train
Loading Test


In [ ]:
train_images, train_labels = shuffle(train_images,train_labels,random_state=25)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(150,150,3)))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5,activation="softmax"))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 150, 150, 16)      208       
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 75, 75, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_53 (Conv2D)          (None, 75, 75, 32)        2080      
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 37, 37, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 37, 37, 64)        8256      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 18, 18, 64)     

In [ ]:
# Compile
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_data_dir = 'C:\\Users\\jacks\\Downloads\\example\\Train'
validation_data_dir = 'C:\\Users\\jacks\\Downloads\\example\\Test'
nb_train_samples = 839
nb_validation_samples = 190
epochs = 50
batch_size = 32

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(300, 300),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(300, 300),
    batch_size=batch_size,
    class_mode='binary')


In [ ]:
history = model.fit(train_images, train_labels, batch_size=32, epochs=100, verbose = 1)

Epoch 1/100
162/162 [==============================] - 91s 554ms/step - loss: 5.8631 - accuracy: 0.4643
Epoch 2/100
162/162 [==============================] - 83s 513ms/step - loss: 0.7998 - accuracy: 0.7101
Epoch 3/100
162/162 [==============================] - 81s 500ms/step - loss: 0.4188 - accuracy: 0.8567
Epoch 4/100
162/162 [==============================] - 82s 503ms/step - loss: 0.2640 - accuracy: 0.9121
Epoch 5/100
162/162 [==============================] - 86s 528ms/step - loss: 0.1955 - accuracy: 0.9367
Epoch 6/100
162/162 [==============================] - 82s 504ms/step - loss: 0.2159 - accuracy: 0.9322
Epoch 7/100
162/162 [==============================] - 81s 502ms/step - loss: 0.1532 - accuracy: 0.9533
Epoch 8/100
162/162 [==============================] - 81s 498ms/step - loss: 0.1065 - accuracy: 0.9674
Epoch 9/100
162/162 [==============================] - 81s 502ms/step - loss: 0.0662 - accuracy: 0.9791
Epoch 10/100
162/162 [==============================] - 82s 507m

In [ ]:
test_loss = model.evaluate(test_images,test_labels)

32/32 [==============================] - 5s 164ms/step - loss: 0.0074 - accuracy: 0.9990


In [ ]:
predictions = model.predict(test_images)
pred_labels = np.argmax(predictions,axis = 1)

In [ ]:
print(classification_report(test_labels,pred_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       213
           1       1.00      1.00      1.00       230
           2       1.00      1.00      1.00       210
           3       1.00      0.99      1.00       190
           4       1.00      1.00      1.00       170

    accuracy                           1.00      1013
   macro avg       1.00      1.00      1.00      1013
weighted avg       1.00      1.00      1.00      1013



In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

In [ ]:
from google.colab import files
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>